## Import dependecies

In [ ]:
import numpy as np
import pandas as pd 
import os
import keras.applications.xception as xception
import tensorflow as tf
import re
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model


## The constansts

In [ ]:
IMAGE_WIDTH = 320    
IMAGE_HEIGHT = 320
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3

base_path = "../data_Set/archive/garbage_classification/"

# Dictionary to save our 13 classes
categories = {0: 'paper', 1: 'cardboard', 2: 'plastic', 3: 'metal', 4: 'trash', 5: 'battery',
              6: 'shoes', 7: 'clothes', 8: 'green-glass', 9: 'brown-glass', 10: 'white-glass',
              11: 'biological', 12:'electronics'}


### image path

In [ ]:
my_img="../data_Set/archive/my_test/x.png"

### preprocess image input

In [ ]:
tmage = load_img(my_img, target_size=(320, 320))
tmage = img_to_array(tmage)
tmage = np.array(tmage)
tmage = np.expand_dims(tmage, axis=0)

## some extra data that i dont understand but are important

In [ ]:
# Add class name prefix to filename. So for example "/paper104.jpg" become "paper/paper104.jpg"
def add_class_name_prefix(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x[:re.search("\d",x).start()] + '/' + x)
    return df

# list conatining all the filenames in the dataset
filenames_list = []
# list to store the corresponding category, note that each folder of the dataset has one class of data
categories_list = []

for category in categories:
    filenames = os.listdir(base_path + categories[category])
    
    filenames_list = filenames_list  +filenames
    categories_list = categories_list + [category] * len(filenames)
    
df = pd.DataFrame({
    'filename': filenames_list,
    'category': categories_list
})

df = add_class_name_prefix(df, 'filename')

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

print('number of elements = ' , len(df))

In [ ]:
#Change the categories from numbers to names
df["category"] = df["category"].replace(categories) 

# We first split the data into two sets and then split the validate_df to two sets
train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)
validate_df, test_df = train_test_split(validate_df, test_size=0.5, random_state=42)

test_df = test_df.reset_index(drop=True)

In [ ]:
test_datagen = image.ImageDataGenerator()

test_generator = test_datagen.flow_from_dataframe(
    dataframe= test_df,
    directory=base_path,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    class_mode="categorical",
    batch_size=1,
    shuffle=False 
)

## Load model

In [ ]:
test_model= tf.keras.models.load_model("model")


In [ ]:
a_prediction = test_model.predict(tmage)


In [ ]:
gen_label_map = test_generator.class_indices
gen_label_map = dict((v,k) for k,v in gen_label_map.items())

In [ ]:
preds = a_prediction.argmax(1)


In [ ]:
preds = [gen_label_map[item] for item in preds]

In [ ]:
preds
